In [1]:
import nltk
nltk.download('punkt')
import re
from gensim import models, corpora
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('brown')
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from bs4 import BeautifulSoup
import re
from sklearn import preprocessing

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


In [2]:
datasource=pd.read_csv('https://raw.githubusercontent.com/Killer2499/Support-Tickets-Topic-Classification/master/Deep%20Learning/latest_ticket_data.csv')
datasource.head()

,Description,Category
0,hi since recruiter lead permission approve req...,Application
1,re expire days hi ask help update passwords co...,Application
2,verification warning hi has got attached pleas...,Application
3,please dear looks blacklisted receiving mails ...,Application
4,dear modules report report cost thank much reg...,Application


In [0]:
features=datasource['Description']
labels=datasource['Category']


In [4]:
cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(cList.keys()))
 
def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)
  
expandContractions("I ain't going anywhere")

'I am not going anywhere'

In [0]:
def clean_text(text):

    
    #Removing comments fom body
    cleaned_text = text.replace(r'From:.*$', '')


    #Expanding the sentence
    cleaned_text = (re.sub("([A-Z]{1}[a-z])"," \\1",i) for i in  cleaned_text)

    #To Lower case
    #cleaned_text = cleaned_text.lower()

    #Removing email ID's
    cleaned_text = (re.sub("[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\\.[a-zA-Z0-9.]+","",i) for i in  cleaned_text)

    #Removing hyperlinks
    cleaned_text = (re.sub("(http:\/\/www\.|https:\/\/www\.|http:\/\/|https:\/\/)?[a-z0-9]+([\-\.]{1}[a-zA-Z0-9]+)*\.[a-z]{2,5}(:[0-9]{1,5})?(\/.*)?","",i) for i in  cleaned_text)

    cleaned_text = (re.sub("http\S+","",i) for i in  cleaned_text)

    #cleaned_text = cleaned_text.replace(r'url= .', '')

    #Replacing Contractions
    cleaned_text = (re.sub("’","'",i) for i in  cleaned_text)

    cleaned_text = (expandContractions(i) for i in  cleaned_text)

    #Removing unnecessary punctuations
    cleaned_text = (re.sub("[\(\)\-\[\]\{\}\;\:\"\\\<\>\/\@\#\$\%\^\&\*\_\~\–]+"," ",i) for i in  cleaned_text)

    cleaned_text = (re.sub("(\?)([a-zA-Z0-9]+)","\\1 \\2",i) for i in  cleaned_text)

    cleaned_text = (re.sub("([A-Za-z]{1,})(\.)([A-Za-z]{2,})","\\1 \\2 \\3",i) for i in  cleaned_text)

    cleaned_text = (re.sub("[\']+","",i) for i in  cleaned_text)

    #cleaned_text = cleaned_text.replace(r'\xa0', ' ')

    cleaned_text = (re.sub("(\.)(\,|\?|\'|\!|\s){1,}","\\1 ",i) for i in  cleaned_text)

    cleaned_text = (re.sub("(!){1}(\s)*(\=)"," not equals ",i) for i in  cleaned_text)

    cleaned_text = (re.sub("(\=)"," equals ",i) for i in  cleaned_text)


    #Replacing _ with " "
    cleaned_text = (re.sub("_"," ",i) for i in  cleaned_text)
   
    
    #Replacing @–
    #cleaned_text = cleaned_text.apply(lambda i: re.sub(r'[@–]','',i))

    #Removing ASCII
    #cleaned_text = cleaned_text.apply(lambda i:  re.sub(r'[^\x00-\x7F]+',' ', i))

    #Replacing multiple '.' with single '.'
    cleaned_text = (re.sub("\s\.+"," .",i) for i in  cleaned_text)

    cleaned_text = (re.sub("\\.+",".",i) for i in  cleaned_text)

    #Replacing . follower by characters with spaces followed by . followed by characters
    cleaned_text = (re.sub("([0-9]+)(\.{1})([^0-9]+)","\\1 \\2 \\3",i) for i in  cleaned_text)


    # remove after regards
    cleaned_text = (re.sub(r'regards.*$','',i) for i in  cleaned_text)
    cleaned_text = (re.sub(r'tel \+.*$','',i) for i in  cleaned_text) 
    cleaned_text = (re.sub(r'fax \+.*$','',i) for i in  cleaned_text)
   
    #print (cleaned_text)
    
    # Remove all the special characters
    cleaned_text = re.sub(r'\W', ' ', ("").join(cleaned_text))
    #print (cleaned_text)
    
    
    # Remove single characters from the start
    cleaned_text = re.sub(r'\^[a-zA-Z]\s+', ' ', cleaned_text) 

    # Substituting multiple spaces with single space
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text, flags=re.I)

    
    # remove all single characters
    cleaned_text= re.sub(r'\s+[a-zA-Z]\s+', ' ', cleaned_text)
    
    
    #Getting to root word 
    #cleaned_text = (lemmatizeCust(i) for i in  cleaned_text)
    
    #Removing stopwords
    #cleaned_text = (removeStopWord(i) for i in  cleaned_text)

    #Replacing muiltple white spaces
    #cleaned_text = (re.sub(" +"," ",i) for i in  cleaned_text)


    

    return(cleaned_text)
  


In [0]:
stopWordlist =[
 'thank',
 'regards',
 'ocp',
 'thanks',
 'hi',
 'hello',
 'team',
 'team',
 'team',
 'team',
 'team',
 'team',
 'team',
 'team',
 'i',               
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 'only',
 'own',
 'same',
 'so',
 'than',
 'too',
 'very',
 's',
 't',
 'can',
 'will',
 'just',
 'don',
 "don't",
 'should',
 "should've",
 'now',
 'd',
 'll',
 'm',
 'o',
 're',
 've',
 'y',
 'ain',
 'aren',
 "aren't",
 'couldn',
 "couldn't",
 'didn',
 "didn't",
 'doesn',
 "doesn't",
 'hadn',
 "hadn't",
 'hasn',
 "hasn't",
 'haven',
 "haven't",
 'isn',
 "isn't",
 'ma',
 'mightn',
 "mightn't",
 'mustn',
 "mustn't",
 'needn',
 "needn't",
 'shan',
 "shan't",
 'shouldn',
 "shouldn't",
 'wasn',
 "wasn't",
 'weren',
 "weren't",
 'won',
 "won't",
 'wouldn',
 "wouldn't",
  "xa0",
  "nbsp"]

In [0]:
def removeStopWord(text):
    string = ""
    for word in text.split(" "):
        if(word in stopWordlist):
            string = string + ""
        else:
            string = string + " " + word
    return string 

In [8]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

porter=PorterStemmer()
def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

x=stemSentence("I am loving it")
print(x)


I am love it 


In [9]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
cleaned_data_train=[]
for sentence in range(0,len(features)):  #Define Your features above
  cleaned_sentence=expandContractions(str(features[sentence]))
  
  cleaned_sentence=clean_text(cleaned_sentence)
  cleaned_sentence=stemSentence(cleaned_sentence)
  cleaned_sentence=removeStopWord(cleaned_sentence)
  
 
  cleaned_data_train.append(cleaned_sentence)
  

In [0]:

from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords 

stop_words = ['in', 'of', 'at', 'a', 'the','an','or','i','and','has','he','will','was','with','is','its','if']

cv_vect = CountVectorizer(binary=True,ngram_range=(1,2),stop_words=stop_words)

cv_vect.fit(features)
cleaned_data_vector= cv_vect.transform(features)
labels = labels.factorize()[0]

In [47]:
print(labels[:100])
#print(labels.factorize()[1])
print(labels.shape)
y = np.bincount(labels[:600])
ii = np.nonzero(y)[0]
for a,b in zip(ii,y[ii]):
  print(a,b)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(3000,)
0 600


**Importing Models**

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold



**Splitting dataset into training and testing set**

**"Random Forest Classifier"**

In [0]:
text_classifier_RF= RandomForestClassifier(n_estimators=200, random_state=0) 
rf_scores=[]
cv = KFold(n_splits=5, random_state=42,shuffle = True)
for train_index, test_index in cv.split(cleaned_data_vector):
    #print("Train Index: ", train_index, "\n")
    #print("Test Index: ", test_index)

    X_train, X_test, y_train, y_test = cleaned_data_vector[train_index], cleaned_data_vector[test_index], labels[train_index], labels[test_index]
    text_classifier_RF.fit(X_train, y_train)
    rf_scores.append(text_classifier_RF.score(X_test, y_test))

In [65]:
print(rf_scores)

[0.72, 0.6983333333333334, 0.735, 0.725, 0.6883333333333334]


**"Linear Support Vector Classifier"**

In [66]:
text_classifier_SVC= LinearSVC(C=1)
svc_scores=[]
cv = KFold(n_splits=5, random_state=42, shuffle=True)

for train_index, test_index in cv.split(cleaned_data_vector):
    #print("Train Index: ", train_index, "\n")
    #print("Test Index: ", test_index)

    X_train, X_test = cleaned_data_vector[train_index], cleaned_data_vector[test_index]
    y_train, y_test =labels[train_index], labels[test_index]
    text_classifier_SVC.fit(X_train, y_train)
    svc_scores.append(text_classifier_SVC.score(X_test, y_test))



/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [67]:
print(sum(svc_scores) / len(svc_scores))
print (svc_scores)


0.7276666666666667
[0.7233333333333334, 0.7466666666666667, 0.73, 0.7083333333333334, 0.73]


**"Support Vector Machine"**

In [0]:
text_classifier_SVM= SVC()
svm_scores=[]
cv = KFold(n_splits=5, random_state=42, shuffle=True)

for train_index, test_index in cv.split(cleaned_data_vector):
    # print("Train Index: ", train_index, "\n")
    # print("Test Index: ", test_index)

    X_train, X_test, y_train, y_test = cleaned_data_vector[train_index], cleaned_data_vector[test_index], labels[train_index], labels[test_index]
    text_classifier_SVM.fit(X_train, y_train)
    svm_scores.append(text_classifier_SVM.score(X_test, y_test))


In [76]:
print(sum(svm_scores) / len(svm_scores))
print(svm_scores)

0.6803333333333333
[0.6866666666666666, 0.6683333333333333, 0.6866666666666666, 0.675, 0.685]


**"Naive Bayes Algorithm"**

In [0]:
from sklearn.naive_bayes import GaussianNB 
text_classifier_NB = GaussianNB() 
nb_scores=[]
cv = KFold(n_splits=5, random_state=42, shuffle=True)

for train_index, test_index in cv.split(cleaned_data_vector):
    #print("Train Index: ", train_index, "\n")
    #print("Test Index: ", test_index)

    X_train, X_test, y_train, y_test = cleaned_data_vector[train_index], cleaned_data_vector[test_index], labels[train_index], labels[test_index]
    text_classifier_NB.fit(X_train.toarray(), y_train)
    nb_scores.append(text_classifier_NB.score(X_test.toarray(), y_test))


In [78]:
print(sum(nb_scores) / len(nb_scores))
print(nb_scores)

0.6976666666666667
[0.7133333333333334, 0.7133333333333334, 0.71, 0.6733333333333333, 0.6783333333333333]


In [0]:
from sklearn.naive_bayes import MultinomialNB
text_classifier_MNB = MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True) 
mnb_scores=[]
cv = KFold(n_splits=5, random_state=42, shuffle=True)

for train_index, test_index in cv.split(cleaned_data_vector):
    #print("Train Index: ", train_index, "\n")
    #print("Test Index: ", test_index)

    X_train, X_test, y_train, y_test = cleaned_data_vector[train_index], cleaned_data_vector[test_index], labels[train_index], labels[test_index]
    text_classifier_MNB.fit(X_train.toarray(), y_train)
    mnb_scores.append(text_classifier_MNB.score(X_test.toarray(), y_test))


In [80]:
print(sum(mnb_scores)/len(mnb_scores))
print(mnb_scores)

0.7443333333333333
[0.7533333333333333, 0.7466666666666667, 0.7683333333333333, 0.7066666666666667, 0.7466666666666667]


**"Logistic Regression Algorithm"**

In [0]:
text_classifier_LR= LogisticRegression(solver='lbfgs', multi_class='multinomial',random_state=1)
lr_scores=[]
cv = KFold(n_splits=5, random_state=42, shuffle=True)

for train_index, test_index in cv.split(cleaned_data_vector):
    #print("Train Index: ", train_index, "\n")
    #print("Test Index: ", test_index)

    X_train, X_test, y_train, y_test = cleaned_data_vector[train_index], cleaned_data_vector[test_index], labels[train_index], labels[test_index]
    text_classifier_LR.fit(X_train, y_train)
    lr_scores.append(text_classifier_LR.score(X_test, y_test))

In [81]:
print(sum(lr_scores)/len(lr_scores))
print(lr_scores)

0.7473333333333334
[0.7483333333333333, 0.755, 0.7616666666666667, 0.7183333333333334, 0.7533333333333333]


In [0]:
text=[ "Sentence_1","Sentence_2" ]
scores=[]

for sentence in range(0,len(text)):
  
  cleaned_input_sentence=expandContractions(str(text[sentence]))
  cleaned_input_sentence=clean_text(cleaned_input_sentence)
  cleaned_input_sentence=stemSentence(cleaned_input_sentence)
  cleaned_input_sentence=removeStopWord(cleaned_input_sentence)
  scores.append(cleaned_input_sentence)
  
  
  
cleaned_input_vector= cv_vect.transform(scores)



In [63]:
text_classifier_LR.predict(cleaned_input_vector)

array([0, 0])

In [0]:
text_classifier_SVC.predict(cleaned_input_vector)

In [0]:
text_classifier_RF.predict(cleaned_input_vector)